### Response to Reviews
## Comparison of quantificaiton methods

In [ ]:
version

In [ ]:
library(tidyverse)
library(ggpubr)
library(circular)

packageVersion("tidyverse")
packageVersion("ggpubr")
packageVersion("circular")

In [ ]:
# scientific notation
# https://stats.biopapyrus.jp/r/ggplot/scientific-notation.html
# https://stackoverflow.com/questions/11610377/how-do-i-change-the-formatting-of-numbers-on-an-axis-with-ggplot

scientific_notation <- function(x) {
     x <- format(x, scientific = TRUE)
     x <- gsub("0e\\+00", "0", x)
     x <- gsub("^(.*)e", "'\\1'e", x)
     x <- gsub("e", "%*%10^", x)
     x <- gsub('\\+', '', x)
     parse(text = x)
}

In [ ]:
df <- read_csv("uni_q.csv", col_types = "icdd")
df <- df %>%
    mutate(pattern = (X1 %/% 8) + 1) %>%
    mutate(filebase = str_c(str_sub(png_file_q, end = -5), "_crop_scale")) %>%
    select(c(pattern, filebase, lightness, complexity))

### Quantification using PAT-GEOM, Chan et al. (2019)
Chan et al. (2019) PAT-GEOM: A software package for the analysis of animal patterns. Methods Ecol Evol 10:591–600. https://doi.org/10.1111/2041-210X.13131

In [ ]:
df_patgeom <- read_csv("uni_q_reps_PAT-GEOM.csv",
                       col_types = "icddddddddddddddddddddddddd")

In [ ]:
options(repr.plot.width = 6, repr.plot.height = 9)

g_lightness <- df %>%
    ggplot(aes(x = pattern, y = lightness, group = pattern)) +
        geom_boxplot() +
        coord_cartesian(ylim = c(0.0, 1.0)) +
        labs(x = "Pattern", y = "Lightness")

g_complexity <- df %>%
    ggplot(aes(x = pattern, y = complexity, group = pattern)) +
        geom_boxplot() +
        coord_cartesian(ylim = c(0.0, 1.0)) +
        labs(x = "Pattern", y = "Complexity")

g_meanAngle <- df_patgeom %>%
    ggplot(aes(x = pattern, y = meanAngle, group = pattern)) +
        geom_boxplot() +
        coord_cartesian(ylim = c(0.0, 180.0)) +
        labs(x = "Pattern", y = "Mean angle") +
        theme_light()

g_meanAspectRatio <- df_patgeom %>%
    ggplot(aes(x = pattern, y = meanAspectRatio, group = pattern)) +
        geom_boxplot() +
        coord_cartesian(ylim = c(1.0, 3.5)) +
        labs(x = "Pattern", y = "Mean aspect ratio") +
        theme_light()

g_meanACS <- df_patgeom %>%
    ggplot(aes(x = pattern, y = meanACS, group = pattern)) +
        geom_boxplot() +
        coord_cartesian(ylim = c(0.0, 300.0)) +
        labs(x = "Pattern", y = "Mean ACS") +
        theme_light()

g_meanSqPx <- df_patgeom %>%
    ggplot(aes(x = pattern, y = meanSqPx, group = pattern)) +
        geom_boxplot() +
        coord_cartesian(ylim = c(0.0, 11000.0)) +
        scale_y_continuous(labels = scientific_notation) +
        labs(x = "Pattern", y = "Mean square pixels") +
        theme_light()

g_randomness <- df_patgeom %>%
    ggplot(aes(x = pattern, y = randomness, group = pattern)) +
        geom_boxplot() + # + coord_cartesian(ylim = c(1500.0, 160000.0))
        scale_y_continuous(labels = scientific_notation) +
        labs(x = "Pattern", y = "Randomness") +
        theme_light()


In [ ]:
pca <- df_patgeom %>%
            select(EFD1:EFD20) %>%
            prcomp(scale = T)

df_pca <- as_tibble(pca$x) %>%
                add_column(ptn = df_patgeom$pattern) %>%
                mutate(pattern = case_when(ptn == 1 ~ "1",
                                           ptn == 2 ~ "2",
                                           ptn == 3 ~ "3",
                                           ptn == 4 ~ "4",
                                           ptn == 5 ~ "5")) %>%
                mutate(pattern = fct_relevel(pattern,
                               "1",
                               "2",
                               "3",
                               "4",
                               "5"))

df_pca

In [ ]:
options(repr.plot.width = 4, repr.plot.height = 3)

palette_hex <- c('#ff6347',  # 1
                 '#ff9d23',  # 2
                 '#ffd700',  # 3
                 '#9dc538',  # 4
                 '#3cb371')  # 5

g_EFA_PCA <- df_pca %>%
    ggplot(aes(x = PC1, y = PC2)) +
    geom_point(aes(color = pattern, fill = pattern, shape = pattern, stroke = pattern), size = 2) +
    scale_color_manual(values = alpha(palette_hex, 0.8)) +
    scale_fill_manual(values = alpha(palette_hex, 0.8)) +
    scale_shape_manual(values = c(21, 24, 4, 24, 21)) +
    scale_discrete_manual(aesthetics = "stroke", values = c(1, 1, 1.5, 1, 1)) +
    labs(x = "EFA-PC1", y = "EFA-PC2",
         color = "Pattern", fill = "Pattern", shape = "Pattern", stroke = "Pattern") +
    theme_light() +
    theme(legend.position = "right")
    

In [ ]:
options(repr.plot.width = 6, repr.plot.height = 9)

plots <- ggarrange(g_lightness,
                   g_complexity,
                   g_meanAngle,
                   g_meanAspectRatio,
                   g_meanACS,
                   g_meanSqPx,
                   g_randomness,
                   g_EFA_PCA,
                   ncol = 2,
                   nrow = 4,
                   # common.legend = TRUE, legend = "bottom",
                   labels = LETTERS)
plots

# plots %>%
#     ggsave(file = "uni_q_reps.pdf", device=cairo_pdf, width = 6, height = 9)